In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

from __future__ import annotations

import os
import re
from typing import List, Dict, Literal, Optional

# LangGraph / LangChain
from typing_extensions import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import AnyMessage, add_messages
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import Document, HumanMessage, AIMessage
from langchain_upstage import ChatUpstage
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings

embeddings_model = UpstageEmbeddings(model="solar-embedding-1-large")

llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
)

def load_menu_docs(data_path: str) -> List[Document]:
    with open(data_path, "r", encoding="utf-8") as f:
        raw = f.read()

    blocks = re.split(r"\n\s*\d+\.\s+", raw)
    docs: List[Document] = []
    for block in blocks:
        block = block.strip()
        if not block:
            continue
        first_line = block.splitlines()[0].strip()
        content = block
        price_match = re.search(r"가격\s*:\s*₩([\d,]+)", block)
        price = price_match.group(1) if price_match else None
        docs.append(
            Document(
                page_content=content,
                metadata={
                    "menu_name": first_line,
                    "price": price,
                },
            )
        )
    return docs


def build_vectorstore(docs: List[Document]) -> FAISS:
    embeddings = embeddings_model  
    vs = FAISS.from_documents(docs, embeddings)
    return vs

class CafeState(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]
    intent: Optional[Literal["menu", "price", "recommend", "other"]]
    results: List[Dict]

def extract_menu_info(doc: Document) -> Dict:
    content = doc.page_content
    menu_name = doc.metadata.get("menu_name", "Unknown")
    price_match = re.search(r"₩([\d,]+)", content)
    desc_match = re.search(r"설명:\s*(.+?)(?:\n|$)", content, re.DOTALL)
    return {
        "name": menu_name,
        "price": f"₩{price_match.group(1)}" if price_match else "가격 정보 없음",
        "description": desc_match.group(1).strip() if desc_match else "설명 없음",
    }


def classify_intent(text: str) -> Literal["menu", "price", "recommend", "other"]:
    t = text.lower()
    # 가격 문의 키워드
    if any(k in t for k in ["가격", "얼마", "비싸", "저렴", "price"]):
        return "price"
    # 추천 키워드
    if any(k in t for k in ["추천", "뭐가 좋아", "인기", "달달", "상큼", "고소", "추천해", "best"]):
        return "recommend"
    # 특정 메뉴/재료 문의 키워드 (메뉴 이름/원료 단서가 포함된 일반 질의)
    if any(k in t for k in ["아메리카노", "라떼", "카푸치노", "마키아토", "콜드브루", "프라푸치노", "녹차", "티라미수", "우유", "시럽", "카페인", "원두"]):
        return "menu"
    # 기본값: 기타
    return "other"

def search_docs(vs: FAISS, user_message: str, intent: str) -> List[Document]:
    if intent == "price":
        return vs.similarity_search("메뉴 가격", k=5)
    elif intent == "recommend":
        docs = vs.similarity_search(user_message, k=3)
        if not docs:
            docs = vs.similarity_search("인기 메뉴", k=3)
        return docs
    elif intent == "menu":
        return vs.similarity_search(user_message, k=4)
    else:
        return vs.similarity_search(user_message, k=4)


def render_price_answer(items: List[Dict]) -> str:
    lines = ["요청하신 가격 정보입니다:"]
    for it in items[:5]:
        lines.append(f"• {it['name']}: {it['price']}")
    return "\n".join(lines)


def render_menu_answer(items: List[Dict]) -> str:
    if not items:
        return "해당 조건에 맞는 메뉴를 찾지 못했어요. 다른 키워드로 말씀해주실래요?"
    top = items[0]
    extra = "\n".join([f"• {i['name']} — {i['price']}" for i in items[1:4]])
    msg = [
        f"[{top['name']}] {top['price']}",
        f"설명: {top['description']}",
    ]
    if extra:
        msg.append("관련 메뉴도 함께 보여드릴게요:")
        msg.append(extra)
    return "\n".join(msg)


def render_recommend_answer(items: List[Dict], user_message: str) -> str:
    t = user_message
    def pick(predicate):
        for it in items:
            if predicate(it):
                return it
        return items[0] if items else None

    if any(k in t for k in ["달달", "달콤", "sweet"]):
        pick_item = pick(lambda x: any(w in x["description"] for w in ["달콤", "카라멜", "바닐라", "휘핑"]))
    elif any(k in t for k in ["가벼", "깔끔", "light"]):
        pick_item = pick(lambda x: any(w in x["description"] for w in ["깔끔", "부담 없이", "블랙"]))
    elif any(k in t for k in ["우유", "부드러", "크리미", "milk", "cream"]):
        pick_item = pick(lambda x: any(w in x["description"] for w in ["우유", "부드", "크리"]))
    else:
        pick_item = items[0] if items else None

    if not pick_item:
        return "요청하신 취향에 맞는 추천을 찾지 못했어요. 키워드를 조금 더 알려주실래요? (예: 달달, 가벼움, 우유)"

    alt = [i for i in items if i["name"] != pick_item["name"]][:3]
    lines = [
        f"이런 메뉴 어떠세요? → {pick_item['name']} ({pick_item['price']})",
        f"설명: {pick_item['description']}",
    ]
    if alt:
        lines.append("함께 고려해볼 만한 메뉴:")
        lines += [f"• {i['name']} — {i['price']}" for i in alt]
    return "\n".join(lines)


def render_fallback_answer(user_message: str) -> str:
    return (
        "도움이 필요하신 주제를 알려주세요! 예) ‘아메리카노 가격’, ‘우유 들어간 메뉴 추천’, ‘달달한 커피 추천’"
    )

class Nodes:
    def __init__(self, vs: FAISS):
        self.vs = vs

    def classify(self, state: CafeState) -> CafeState:
        user_msg = state["messages"][-1]["content"] if isinstance(state["messages"][-1], dict) else state["messages"][-1].content
        intent = classify_intent(user_msg)
        state["intent"] = intent
        return state

    def retrieve(self, state: CafeState) -> CafeState:
        user_msg = state["messages"][-1]["content"] if isinstance(state["messages"][-1], dict) else state["messages"][-1].content
        intent = state.get("intent") or classify_intent(user_msg)
        docs = search_docs(self.vs, user_msg, intent)
        items = [extract_menu_info(d) for d in docs]
        state["results"] = items
        return state

    def respond(self, state: CafeState) -> CafeState:
        user_msg = state["messages"][-1]["content"] if isinstance(state["messages"][-1], dict) else state["messages"][-1].content
        intent = state.get("intent", "other")
        items = state.get("results", [])
        if intent == "price":
            answer = render_price_answer(items)
        elif intent == "menu":
            answer = render_menu_answer(items)
        elif intent == "recommend":
            answer = render_recommend_answer(items, user_msg)
        else:
            answer = render_fallback_answer(user_msg)

        state["messages"].append(AIMessage(content=answer))
        return state


def build_graph(vs: FAISS):
    nodes = Nodes(vs)
    graph = StateGraph(CafeState)
    graph.add_node("classify", nodes.classify)
    graph.add_node("retrieve", nodes.retrieve)
    graph.add_node("respond", nodes.respond)

    graph.add_edge(START, "classify")
    graph.add_edge("classify", "retrieve")
    graph.add_edge("retrieve", "respond")
    graph.add_edge("respond", END)

    return graph.compile()

def run_repl(app):
    print("\n💬 카페 챗봇이 준비됐어요. (끝내기: /quit)\n")
    state: CafeState = {"messages": [], "intent": None, "results": []}
    while True:
        user = input("> ").strip()
        if not user:
            continue
        if user.lower() in {"/q", "/quit", "/exit"}:
            print("안녕히 가세요! ☕")
            break
        state["messages"].append(HumanMessage(content=user))

        state = app.invoke(state)

        print(state["messages"][-1].content)

if __name__ == "__main__":

    data_path = os.getenv("MENU_DATA_PATH", "../../data/cafe_menu_data.txt")
    if not os.path.exists(data_path):
        alt = "/mnt/data/cafe_menu_data.txt"
        data_path = alt
    docs = load_menu_docs(data_path)
    vs = build_vectorstore(docs)
    app = build_graph(vs)
    run_repl(app)


gs
zQ


c:\Users\hia\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-unC8AIy--py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



💬 카페 챗봇이 준비됐어요. (끝내기: /quit)

이런 메뉴 어떠세요? → 티라미수 (₩7,500)
설명: 이탈리아 전통 디저트로 마스카포네 치즈와 에스프레소에 적신 레이디핑거를 층층이 쌓아 만들었습니다. 부드럽고 달콤한 맛이 특징이며, 코코아 파우더로 마무리하여 깊은 풍미를 자랑합니다.
함께 고려해볼 만한 메뉴:
• 바닐라 라떼 — ₩6,000
• 프라푸치노 — ₩7,000
